# Example notebook to show some features of prodimopy

If you launched this notebook via a binder url please note that all data is lost after the end of the session.
However, like in any notebook you can download the files.

In [ ]:
%matplotlib inline
import urllib.request
import tarfile

# module to read a ProDimo model
import prodimopy.read as pread
# module for plotting a ProDiMo model
import prodimopy.plot as pplot

import matplotlib.pyplot as plt

# Download and read in a DIANA Standard model

For more details on DIANA and more DIANA standard models see https://dianaproject.wp.st-andrews.ac.uk/ and http://www-star.st-and.ac.uk/~pw31/DIANA/DIANAstandard/

In [ ]:
# Download and extract the Model tarfile into a Directory ... this might take a while ... be patient
target="DMTau"
modeltarfile=urllib.request.urlretrieve("http://www-star.st-and.ac.uk/~pw31/DIANA/DIANAstandard/"+target+"_ModelOutput.tgz")[0]
tarfile.open(modeltarfile).extractall(target)

# now read the model data with prodimopy
model=pread.read_prodimo(target)

# Plot the data within the notebook

In [ ]:
# load a nice stylesheet for plotting within the notebook
plt.style.use(["prodimo.mplstyle","prodimonb.mplstyle"])
# Create a Plot oject for plotting directly to the notebook

pp=pplot.Plot(None)

fig=pp.plot_NH(model)
# you can still save the figure to a pdf file if you want
fig.savefig("plotNH.pdf")

In [ ]:
# and example contour plot
fig=pp.plot_cont(model,model.rhog,label=r"$\mathrm{log\,\rho\,[g cm^{-3}]}$",zlim=[1.e-20,1.e-10],extend="both")

# Write a number of plots directly to a singl pdf file

In [ ]:
# Put a couple of plots to a single pdf
from matplotlib.backends.backend_pdf import PdfPages

# switch back to style better suited for pdf
plt.style.use("prodimo.mplstyle")

with PdfPages(target+".pdf") as pdf:
    pp=pplot.Plot(pdf,title=model.name)
      
    # This plots the Stellar spectrum  
    pp.plot_starspec(model,ylim=[1.e-1,1.e11])  
    
    if model.sed is not None: 
        pp.plot_sed(model)
      
    pp.plot_dust_opac(model,ylim=[1.e-1,None])
      
    pp.plot_NH(model)
    
    # here the default contour plots are used not very fancy currently. 
    # you can use latex for the labels!
    # note most of the parameters are optional
    pp.plot_cont(model, model.nHtot, r"$\mathrm{n_{<H>} [cm^{-3}]}$")  
  
    pp.plot_cont(model, model.nHtot, r"$\mathrm{n_{<H>} [cm^{-3}]}$",contour=True,zr=False,xlog=False,ylog=False,
                   zlim=[1.e4,None], extend="both")  
    pp.plot_cont(model, model.rhod, r"$\mathsf{\rho_{dust} [g\;cm^{-3}]}$",zr=True,xlog=True,ylog=False,
                    zlim=[1.e-25,None],extend="both")
    pp.plot_cont(model, model.nd, r"$\mathrm{n_{dust} [cm^{-3}]}$",contour=True,zr=True,xlog=True,ylog=False,
                    zlim=[1.e-8,None],extend="both")
    
    pp.plot_cont(model, model.tg, r"$\mathrm{T_{gas} [K]}$",contour=True,
                    zlim=[5,5000],extend="both")

    pp.plot_cont(model, model.tg, r"$\mathrm{T_{dust} [K]}$",contour=True,
                    zlim=[5,1500],extend="both")
    
    # this plots the vertical abundances at different radii for different species
    rs=[10]
    species=["CO","HCO+","e-","S+"]
    for r in rs:
        pp.plot_abunvert(model, r, species,ylim=[3.e-15,3.e-3])
      
    # plots the average abundances of the above species as a function of radius
    pp.plot_avgabun(model, species,ylim=[3.e-15,3.e-3])
    
    # contour plots for a couple of species
    species=["CO","H2O","HCO+","HN2+"]
    for spname in species:
        pp.plot_abuncont(model, spname, zr=True,xlog=True,ylog=False, zlim=[1.e-4,3.e-15],extend="both")
  
    pp.plot_line_origin(model,[["CO",1300.0]],
                        model.nmol[:,:,model.spnames["CO"]],zlim=[1.e-3,1.e8],ylim=[None,0.5],
                        extend="both",showContOrigin=True)

# Extract some data and put it into a file

In [51]:
import numpy

# get the dust temperature in the midplane
tdust=model.td[:,0]
# get the r coordinate in the midplane
r=model.x[:,0]

numpy.savetxt(target+"tmid.txt",numpy.column_stack((r,tdust)),header="midplane dust temperature as a function of radius\n r [au] Tdust [K]")